In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# A Symbolic Calculator with Lezer \& RecursiveSet

This notebook demonstrates the implementation of a **symbolic calculator** that parses, analyzes, and evaluates assignments and arithmetic expressions.

The distinctive feature of this implementation is the use of **value semantics** for the Abstract Syntax Tree (AST) and the environment. Instead of ordinary JavaScript objects, we utilize the `recursive-set` library to enable structural equality and hash-based lookup.

## Language Grammar

The grammar for our calculator language is formally defined as follows:

$$
\begin{array}{lcl}
\texttt{stmnt} & \rightarrow & \texttt{IDENTIFIER}\; \texttt{':='}\; \texttt{expr}\; \texttt{';'} \\
& \mid & \texttt{expr}\; \texttt{';'} \\[0.3cm]
\texttt{expr} & \rightarrow & \texttt{expr}\; \texttt{'+'}\; \texttt{product} \\
& \mid & \texttt{expr}\; \texttt{'-'}\; \texttt{product} \\
& \mid & \texttt{product} \\[0.3cm]
\texttt{product} & \rightarrow & \texttt{product}\; \texttt{'*'}\; \texttt{factor} \\
& \mid & \texttt{product}\; \texttt{'/'}\; \texttt{factor} \\
& \mid & \texttt{factor} \\[0.3cm]
\texttt{factor} & \rightarrow & \texttt{'('}\; \texttt{expr}\; \texttt{')'} \\
& \mid & \texttt{'+'}\; \texttt{factor} \\
& \mid & \texttt{'-'}\; \texttt{factor} \\
& \mid & \texttt{NUMBER} \\
& \mid & \texttt{IDENTIFIER}
\end{array}
$$

This grammar ensures proper operator precedence (multiplication and division before addition and subtraction) through its hierarchical structure. Left recursion in the $\texttt{expr}$ and $\texttt{product}$ rules guarantees left-associative evaluation.

## Architecture & Design Philosophy

The interpreter pipeline consists of four distinct stages:

1. **Grammar Definition:** We declaratively specify the language syntax using **Lezer**.
2. **Runtime Parsing:** `buildParser` generates an efficient LR parser from the grammar.
3. **AST Transformation (Generic Mapper):** The flat Lezer tree is transformed into a typed AST using a declarative mapping configuration (`cstToAST`).
4. **Semantic Analysis & Evaluation:** We employ `RecursiveSet` for variable analysis and `RecursiveMap` as a variable store.

### Why Tuple and RecursiveSet?

In classical AST implementations, two nodes `new BinOp("+", 1, 2)` and `new BinOp("+", 1, 2)` are **not equal** under JavaScript's default reference equality.

Our `ASTNode` base class solves this by wrapping the data in a `Tuple`, providing **Structural Equality**:
- **Identity:** Two AST nodes with identical content are logically equivalent.
- **Hashing:** AST nodes can serve as keys in maps or elements in sets.

In [ ]:
import { buildParser } from '@lezer/generator';
import { TreeCursor } from '@lezer/common';
import { LRParser } from '@lezer/lr'
import { RecursiveSet, RecursiveMap } from "recursive-set";

## Domain Types (The AST)

We define the language constructs as classes inheriting from `ASTNode`. This ensures type safety while maintaining the structural equality properties provided by the underlying `Tuple`.

In [ ]:
import { 
    AST, NumNode, VarNode, BinaryExpr, AssignNode, ExprStmtNode, 
    cstToAST, ParserConfig, ast2dot 
} from "./AST2Dot";

**Type Hierarchy:**

$$
\begin{array}{lcl}
\texttt{Expr} & ::= & \texttt{Num} \mid \texttt{Var} \mid \texttt{BinOp} \\
\texttt{Stmnt} & ::= & \texttt{Assign} \mid \texttt{ExprStmnt}
\end{array}
$$

In [ ]:
type Expr = NumNode | VarNode | BinaryExpr;
type Stmnt = AssignNode | ExprStmtNode;

## Grammar Definition (Lezer)

The grammar is specified declaratively using Lezer's syntax. Because Lezer is an LR-based parser, it can handle left-recursive rules natively. 

Notice that this implementation matches our formal mathematical definition **almost 1:1**. Each branch of the formal grammar is explicitly represented, ensuring that the resulting parse tree reflects the exact structure of our language.

- **Direct Mapping:** Every rule from the EBNF is translated into a corresponding Lezer rule.
- **Left Recursion:** By writing `expr { BinaryExpr { expr Plus product } ... }`, we ensure that the parser naturally groups operations from left to right (Left-Associativity).
- **Precedence:** The hierarchy (Factor → Product → Expression) automatically enforces that multiplication and division are evaluated before addition and subtraction.
- **Semantic vs. Structural Tokens:** We explicitly name tokens like `Plus` or `Mul` because they carry semantic meaning required for the AST construction. Purely structural markers like `"("` or `";"` remain anonymous (literal strings) as they serve only to guide the parser structure and are discarded during the transformation.

In [ ]:
const grammarDefinition = `
    @top Program { stmnt }

    stmnt {
        Assign { Identifier ":=" expr ";" } |
        ExprStmnt { expr ";" }
    }

    expr {
        BinaryExpr { expr Plus product } |
        BinaryExpr { expr Minus product } |
        product
    }

    product {
        BinaryExpr { product Mul factor } |
        BinaryExpr { product Div factor } |
        factor
    }

    factor {
        ParenExpr { "(" expr ")" } |
        UnaryExpr { Plus factor } |
        UnaryExpr { Minus factor} |
        Number |
        Identifier
    }

    @tokens {
        Number { ("0" | $[1-9] $[0-9]*) ("." $[0-9]*)? }
        Identifier { $[a-zA-Z] $[a-zA-Z0-9_]* }
        Plus { "+" }
        Minus { "-" }
        Mul { "*" }
        Div { "/" }
        space { $[ \t\r\n]+ }
        "(" ")" ";" ":="
    }

    @skip { space }
`;

const parser: LRParser = buildParser(grammarDefinition);

### Tokenization Helper

In Lezer, tokenization is implicitly integrated into the parsing process. However, to understand how the raw input is broken down before the grammatical structure is applied, we can simulate a standalone scanner.

We traverse the tree and filter **only** for the terminal types defined in our `@tokens` block and errors `'⚠'`, ignoring all structural grammar nodes (like `BinaryExpr` or `stmnt`).

In [ ]:
function tokenizeCalc(input: string): string[] {
    const tree = parser.parse(input);
    const cursor = tree.cursor();
    const tokens: string[] = [];
    
    const validTokens = new Set([
        "Number", "Identifier", 
        "Plus", "Minus", "Mul", "Div", 
        "(", ")", ";", ":=", "⚠"
    ]);

    do {
        if (validTokens.has(cursor.name)) {
            const content = input.slice(cursor.from, cursor.to);
            let name = cursor.name;
            if (name === ":=") name = "AssignOp";
            if (name === ";")  name = "Semicolon";
            if (name === "(")  name = "LParen";
            if (name === ")")  name = "RParen";
            tokens.push(`${name}('${content}')`);
        }
    } while (cursor.next());
    
    return tokens;
}

Example: 

In [ ]:
tokenizeCalc("x := 3 + 4 ^ * 2;");

## Parser Logic (Generic CST to AST Transformation)

Instead of writing a monolithic parser function with complex switch-statements, we use a generic `cstToAST` mapper. This separates the traversal logic from the AST construction.

### The Configuration Logic

The `ParserConfig` object declaratively defines how Lezer's Concrete Syntax Tree (CST) maps to our Abstract Syntax Tree (AST).

**Key Features:**
- **Implicit Nodes:** Rules starting with lowercase letters in Lezer (like `expr`) are transparent. We only need rules for the explicit nodes like `BinaryExpr` or `Assign`.
- **Sugar Removal:** We add tokens like `(`, `)`, and `;` to the `ignore` set, so they don't clutter our children arrays.
- **Desugaring:** The `UnaryExpr` rule demonstrates a simple transformation: it converts syntactic sugar like `-x` directly into the arithmetic operation `-1 * x`.

In [ ]:
const config: ParserConfig = {
    ignore: new Set(["(", ")", ";", ":="]),
    
    rules: {
        "Number": (_, text) => new NumNode(parseFloat(text)),
        "Identifier": (_, text) => new VarNode(text),
        "Plus": () => new VarNode("+"), "Minus": () => new VarNode("-"),
        "Mul": () => new VarNode("*"),  "Div": () => new VarNode("/"),
        "Assign": (children) => {
            const [target, value] = children;
            if (target instanceof VarNode) {
                return new AssignNode(target.name, value);
            }
            throw new Error("Invalid Assignment Target");
        },
        "ExprStmnt": ([expr]) => new ExprStmtNode(expr),
        "Program": ([stmt]) => stmt,
        "BinaryExpr": ([left, op, right]) => {
            if (op instanceof VarNode) {
                return new BinaryExpr(left, op.name, right);
            }
            throw new Error("Expected Operator in BinaryExpr");
        },
        "ParenExpr": ([inner]) => inner,
        "UnaryExpr": ([op, val]) => {
            if (op instanceof VarNode && op.name === "-") {
                return new BinaryExpr(new NumNode(-1), "*", val);
            }
            return val;
        },
        "⚠": (_, text) => { 
            throw new Error(`Syntax Error: Unexpected token '${text}'`); 
        }
    }
};

### 3. The Public API

Finally, `parseStmnt` acts as the type-safe entry point for our calculator. It coordinates the Lezer parser and the transformation mapper.
It also includes a runtime check to ensure that the parsed result is indeed a valid statement (Assignment or Expression), preventing invalid inputs from crashing the interpreter later.

In [ ]:
function parseStmnt(input: string): Stmnt {
    const tree = parser.parse(input);
    const ast = cstToAST(tree.cursor(), input, config);
    if (ast instanceof AssignNode || ast instanceof ExprStmtNode) {
        return ast;
    }
    throw new Error(`Expected a statement, got: ${ast.constructor.name}`);
}

## Static Analysis

Instead of a simple `Record<string, number>`, we use `RecursiveMap` for the environment.

**Advantages:**

- **Efficiency:** Optimized hashing for string keys
- **Deterministic Output:** `env.toString()` produces sorted, deterministic output of the storage state

Additionally, we implement `collectVars`, which collects all used variables from an expression into a `RecursiveSet<string>`. This demonstrates how easily set operations can be performed with the library.

In [ ]:
function collectVars(node: AST): RecursiveSet<string> {
    const vars = new RecursiveSet<string>();
    
    function visit(n: AST) {
        if (n instanceof VarNode) vars.add(n.name);
        else if (n instanceof AssignNode) { vars.add(n.id); visit(n.expr); }
        else if (n instanceof ExprStmtNode) visit(n.expr);
        else if (n instanceof BinaryExpr) { visit(n.left); visit(n.right); }
    }
    
    visit(node);
    return vars;
}

## Operational Semantics

The evaluation phase defines the semantic meaning of our AST nodes through mathematical operations on the abstract machine state.

### Expression Evaluation

The evaluation function $\mathcal{E}: \texttt{Expr} \times \texttt{Env} \to \mathbb{R}$ is defined recursively:

$$
\begin{array}{lcl}
\mathcal{E}(\texttt{Num}(n), \rho) & = & n \\[0.2cm]
\mathcal{E}(\texttt{Var}(x), \rho) & = & \rho(x) \\[0.2cm]
\mathcal{E}(\texttt{BinOp}(\oplus, e_1, e_2), \rho) & = & \mathcal{E}(e_1, \rho) \oplus \mathcal{E}(e_2, \rho)
\end{array}
$$

where $\rho: \texttt{String} \to \mathbb{R}$ represents the environment and $\oplus \in \{+, -, \times, \div\}$.

In [ ]:
type Env = RecursiveMap<string, number>;

function evalExpr(e: AST, env: Env): number {
    if (e instanceof NumNode) return e.value;
    
    if (e instanceof VarNode) {
        const val = env.get(e.name);
        if (val === undefined) throw new Error(`Undefined variable: ${e.name}`);
        return val;
    }
    
    if (e instanceof BinaryExpr) {
        const l = evalExpr(e.left, env);
        const r = evalExpr(e.right, env);
        switch (e.op) {
            case "+": return l + r;
            case "-": return l - r;
            case "*": return l * r;
            case "/": return l / r;
            default: throw new Error(`Unknown op: ${e.op}`);
        }
    }
    throw new Error(`Cannot evaluate node type: ${e.constructor.name}`);
}

### Statement Execution

The execution function $\mathcal{S}: \texttt{Stmnt} \times \texttt{Env} \to \mathbb{R}$ produces a result and updates the environment via side effects:

$$
\begin{array}{lcl}
\mathcal{S}(\texttt{Assign}(x, e), \rho) & = & v \quad \text{where } v = \mathcal{E}(e, \rho) \text{ and } \rho \text{ is updated to } \rho[x \mapsto v] \\[0.2cm]
\mathcal{S}(\texttt{ExprStmnt}(e), \rho) & = & \mathcal{E}(e, \rho)
\end{array}
$$

In [ ]:
function exec(s: Stmnt, env: Env): number {
    if (s instanceof AssignNode) {
        const val = evalExpr(s.expr, env);
        env.set(s.id, val);
        return val;
    }
    if (s instanceof ExprStmtNode) {
        return evalExpr(s.expr, env);
    }
    return 0;
}

Set `inputProgram`for the Calculator:

In [ ]:
const inputProgram = `
x := 10 + 5 * 2;
y := (x - 5) / 3;
z := -y * 2;
z + 100;
`;

## Main Execution Loop

The main function orchestrates the Read-Eval-Print Loop (REPL):

In [ ]:
const env = new RecursiveMap<string, number>();

const lines = inputProgram
    .split("\n")
    .map((l) => l.trim())
    .filter((l) => l.length > 0);

console.log("--- Calculation Start ---");

for (const line of inputProgram.split("\n")) {
    const l = line.trim();
    if (!l) continue;

    try {
        const ast = parseStmnt(l);
        const res = exec(ast, env);
        const vars = collectVars(ast);
        console.log(`Code:   ${l}`);
        console.log(`AST:    ${ast}`);
        console.log(`Vars:   ${vars}`);
        console.log(`Result: ${res}`);
        console.log("-".repeat(30));
        
    } catch (e) {
        console.error(`Error: ${(e as Error).message}`);
    }
}

console.log("\n--- Final Environment ---");
env;